In [1]:
#Iván Andrés Trujillo Abella 
#ivantrujillo1229@gmail.com
#Consolidate Thesis

In [2]:
## Own functions
def error_in_date(x, format_date= "%Y-%m-%d"):
    try:
        datetime.strptime(x,format_date)
        return False
    except:
        return True

def clean_date(x):
    return pd.to_datetime(x[0:10], format= "%Y-%m-%d")    
    
def cleantime(x):
    if type(x)==str:
        return x.replace("00:00:00", "").replace(" ", "")
    else:
        return x

def top_cat(k, cat, df):
    otro = []
    cat_relative_dict =  df[cat].value_counts(normalize=True).to_dict()
    for cat in cat_relative_dict:
        if cat_relative_dict[cat]<=k:
            otro.append(cat)
    return otro


def bankyear(df_):
    df = df_.copy()
    years = [2016, 2017, 2018, 2019]
    df['bankrupt-year'] = 0
    for firm in df.index:
        unchanged = True
        for year in years:
            if df.loc[firm, 'evento'+str(year)]==1 and unchanged==True:
                df.loc[firm,'bankrupt-year']= year
                unchanged = False
    return df['bankrupt-year']



def new_firms(current_df, next_df, key):
    firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
    firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
    return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])


# Now the control are taken from the previous year (in this case is for year)
def varInEvent(time_base, time_last, time_target, prefix, var_time_event, df_, lag=1, mean=True):
    """ put time_target  = 0 if you use mean=True"""
    df = df_.copy()
    df['var*'] = np.nan
    for firm in df.index:
        time_ocurrence = df.loc[firm, var_time_event]
        if time_ocurrence - lag >= time_base:
            df.loc[firm, 'var*'] = df.loc[firm, prefix + str(time_ocurrence-lag)]
        if mean==True:
            to_mean =  [prefix+str(time) for time in range(time_base, time_last+1)]
            df['var*'] =np.where(df[var_time_event]==0, df[to_mean].mean(axis=1), df['var*'])
        else:
            df['var*'] =np.where(df[var_time_event]==0, df[prefix+str(time_target-lag)], df['var*'])
    return df['var*']



def check_vars(vars_to_check, df):
    not_in = []
    for var in vars_to_check:
        if var not in df.columns:
            not_in.append(var)  
    return not_in

def ratios(df_, year):
    df = df_.copy()
  # Marge de ganancia bruta (MGB)
  # Ganancia bruta / ingresos de actividades ordianrias
    df['MGB'] = df['Ganancia bruta'] / df['Ingresos de actividades ordinarias']
  # Remember that ganancia bruta is:
    df['Ingresos de actividades ordinarias'] - df['Costo de ventas']
  ## Margen de ganancia Neta (MGN)
  #df['Ganancia (pérdida) antes de impuestos'] - df['Ingreso (gasto) por impuestos'] #esto es el resultado del periodo
    #df[ 'Ganancia (pérdida)'] # que en la base resultado del periodo es esta variable
    df['MGN'] = df[ 'Ganancia (pérdida)'] /  df['Ingresos de actividades ordinarias']
  # Rendimiento del patrimonio (ROE)
    df['ROE'] = df['Ganancia (pérdida)']/ df['Patrimonio total']
  # Rendimiento del Activo (ROA)
    df['ROA'] = df['Ganancia (pérdida)']/ df['Total de activos']
  ## NIVEL DE ENDEUDAMIENTO (NE)
    df['NE'] = df['Total pasivos'] / df['Total de activos']
  ## Concenttración de Pasivos a corto plazo (PCP)
    df['PCP'] = df['Pasivos corrientes totales'] / df['Total pasivos']
  ## Endeudamiento Financiero (EF)
  ## Crear primero otros pasivos financieros
    df['Otros pasivos financieros'] = df['Otros pasivos financieros corrientes'] + df['Otros activos no financieros corrientes']
    df['EF'] = df['Otros pasivos financieros'] /  df['Ingresos de actividades ordinarias']
  ## Impacto de la carga financiera (IF)
    df['IF'] = df['Costos financieros']/ df['Ingresos de actividades ordinarias']
  ## ALTMAN
    df['Ax1'] = (df['Activos corrientes totales'] - df['Pasivos corrientes totales'])/df['Total de activos']
    df['Ax2'] = (df['Ganancias acumuladas'] / df['Total de activos'])
    return df


In [3]:
cd C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets

C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets


In [4]:
import pandas as pd
import numpy as np

In [5]:
# Loading datasets
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
      dfs['df'+str(year)]  = pd.read_excel('Cáratula-Pymes-'+str(year)+'.xlsx')

In [6]:
for year in years:
    print('#'*10)
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)] =  dfs['df'+str(year)][-pd.isnull(dfs['df'+str(year)]['NIT'])]
    print(dfs['df'+str(year)].shape , year)
    dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
    dfs['df'+str(year)] = dfs['df'+str(year)][['evento'+str(year), 'NIT' ,'Clasificación Industrial Internacional Uniforme Versión 4 A.C']]
    print(dfs['df'+str(year)]['NIT'].duplicated().sum())
    print(dfs['df'+str(year)].shape , year)

##########
(17799, 31) 2016
(17723, 31) 2016
(17701, 32) 2016
(17701, 32) 2016
0
(17701, 3) 2016
##########
(14696, 32) 2017
(14656, 32) 2017
(14639, 33) 2017
(14638, 33) 2017
0
(14638, 3) 2017
##########
(14275, 34) 2018
(14229, 34) 2018
(14212, 35) 2018
(14210, 35) 2018
1
(14210, 3) 2018
##########
(19185, 36) 2019
(19140, 36) 2019
(19128, 37) 2019
(19125, 37) 2019
0
(19125, 3) 2019


In [7]:
dfs['df2016']

,evento2016,NIT,Clasificación Industrial Internacional Uniforme Versión 4 A.C
22,0,800000296,F4290 - Construcción de otras obras de ingenie...
23,0,800000313,C3290 - Otras industrias manufactureras n.c.p.
24,0,800000439,G4690 - Comercio al por mayor no especializado
25,0,800000457,M7310 - Publicidad
26,0,800000835,A0114 - Cultivo de tabaco
...,...,...,...
17794,0,900984398,K6492 - Actividades financieras de fondos de e...
17795,0,900991284,K6424 - Actividades de las cooperativas financ...
17796,0,900995838,K6499 - Otras actividades de servicio financie...
17797,0,901035582,K6613 - Otras actividades relacionadas con el ...


In [8]:
dfs['df2019']['evento2019'].value_counts() # Here there are 876 firms,  but since declarate the process.

evento2019
0    18249
1      876
Name: count, dtype: int64

In [9]:
for year in range(2016,2020):
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())

evento2016
0    17193
1      508
Name: count, dtype: int64
evento2017
0    14019
1      619
Name: count, dtype: int64
evento2018
0    13445
1      765
Name: count, dtype: int64
evento2019
0    18249
1      876
Name: count, dtype: int64


In [10]:
full = dfs['df2016'].copy()
for year in range(2017,2020):
    clients= new_firms(full,dfs['df'+str(year)], 'NIT')
    full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)
    full = pd.concat([full, clients])

In [11]:
full['NIT'].duplicated().sum()

1

In [12]:
full['evento2019'].value_counts()

evento2019
0.0    18250
1.0      876
Name: count, dtype: int64

In [13]:
"""
H = pd.DataFrame( [[1,2,3,4,5], [np.nan, np.nan, np.nan, np.nan, np.nan],
                   [np.nan, np.nan, np.nan, np.nan, 1]  ,  [0, np.nan, np.nan, np.nan, np.nan] ])
H.sum(axis=1, numeric_only=True, min_count=1) # This is very important... 
"""

'\nH = pd.DataFrame( [[1,2,3,4,5], [np.nan, np.nan, np.nan, np.nan, np.nan],\n                   [np.nan, np.nan, np.nan, np.nan, 1]  ,  [0, np.nan, np.nan, np.nan, np.nan] ])\nH.sum(axis=1, numeric_only=True, min_count=1) # This is very important... \n'

In [14]:
full[['evento2016', 'evento2017', 'evento2018', 'evento2019']].sum(axis=1, min_count=1).value_counts(dropna=False)

0.0    25932
1.0      508
2.0      300
4.0      247
3.0      224
Name: count, dtype: int64

In [15]:
full = full[-full['NIT'].duplicated()].reset_index(drop=True)
full['time-event'] =bankyear(full)
full['time-event'].value_counts(dropna=False)

time-event
0       25931
2016      508
2019      282
2018      257
2017      232
Name: count, dtype: int64

In [16]:
## Draw firmst that enter to process in 2016 to avoid bias in indicators...
#full = full[full['evento2016']!=1]
#print(full['time-event'].value_counts())

In [17]:
full['event'] = (full['time-event']!=0).astype('int')

In [18]:
full['event'].value_counts()

event
0    25931
1     1279
Name: count, dtype: int64

In [19]:
ERIS={}
EFSS={}
for year in range(2016,2019):
        #loops in ERI
    ERIS['eri'+str(year)] = pd.read_excel('ERI-Pymes-'+str(year)+'.xlsx')
    ERIS['eri'+str(year)]['Fecha'] = pd.to_datetime( ERIS['eri'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Fecha'].dt.month==12]
    if year==2016:
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.year==2016]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.month==12]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.day==31]
    else:
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Periodo']=='Periodo Actual']
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][pd.to_datetime(ERIS['eri'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

    #ERIS['eri'+str(year)][-ERIS['eri'+str(year)]['NIT'].duplicated()].reset_index(drop=True)

    # loops in efs
    EFSS['efs'+str(year)] = pd.read_excel('ESF-Pymes-'+str(year)+'.xlsx')
    if year == 2016:
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.year==2016]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.month==12]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.day==31]
    else:
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][EFSS['efs'+str(year)]['Periodo']=='Periodo Actual']
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.month==12]
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

## Lines to debug...
#EFSS['efs2018']['NIT'].duplicated(keep=False).sum()
#EFSS['efs2018'][EFSS['efs2018']['NIT'].duplicated(keep=False)].to_excel('2018_review.xlsx')

In [20]:
# We can integrate both statements...

FULLS = {}
for year in range(2016,2019):
    # Number of fimrs by year in each statement!
    print('--'*10)
    print(year,EFSS['efs'+str(year)].shape[0] ,  ERIS['eri'+str(year)].shape[0])
    FULLS['full'+str(year)] = EFSS['efs'+str(year)].merge(ERIS['eri'+str(year)], on='NIT', how='outer',
                                                          indicator = True, suffixes=(None, 'to_remove') )
    duplicated = [var for var in FULLS['full'+str(year)].columns if 'to_remove' in var]
    FULLS['full'+str(year)].drop(columns=duplicated, inplace=True)
    print('--'*10)
    print(FULLS['full'+str(year)]['_merge'].value_counts(dropna=False))

--------------------
2016 17777 17777
--------------------
_merge
both          17777
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2017 14679 14679
--------------------
_merge
both          14679
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2018 14257 14257
--------------------
_merge
both          14257
left_only         0
right_only        0
Name: count, dtype: int64


In [21]:
#FULLS['full2018'].columns

In [22]:
raw_X = ['NIT','Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 
         'Patrimonio total','Total pasivos', 'Total de activos', 'Ganancias acumuladas',  
         'Otros pasivos financieros corrientes','Pasivos corrientes totales', 'Costos financieros', 
         'Otros activos no financieros corrientes', 'Activos corrientes totales']

In [23]:
# Objective : rectificy if the variable exist in all datasets with the name..
# must be appear only empty lists
for year in range(2016,2019):
    print(check_vars(raw_X, FULLS['full'+str(year)]))
    FULLS['full'+str(year)] = FULLS['full'+str(year)][raw_X]
    FULLS['full'+str(year)].columns = ['NIT'] + [col+'-'+str(year) for col in  FULLS['full'+str(year)].columns if col!='NIT']  

[]
[]
[]


In [24]:
full = full.copy()
for year in range(2016,2019):
    full = full.merge(FULLS['full'+str(year)], on='NIT', how='outer', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)

In [26]:
print(full[full['event'].isnull()].shape)
a = set(full[full['event'].isnull()]['NIT'].unique())
b = set(dfs['df2016']['NIT'].unique())
c = list(b & a)
print(len(c)) ## 

(63, 47)
0


In [27]:
full[full['event'].isnull()]

,evento2016,NIT,Clasificación Industrial Internacional Uniforme Versión 4 A.C,evento2017,evento2018,evento2019,time-event,event,Ganancia bruta-2016,Ganancia (pérdida)-2016,...,Costo de ventas-2018,Patrimonio total-2018,Total pasivos-2018,Total de activos-2018,Ganancias acumuladas-2018,Otros pasivos financieros corrientes-2018,Pasivos corrientes totales-2018,Costos financieros-2018,Otros activos no financieros corrientes-2018,Activos corrientes totales-2018
27210,NaN,800176850.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,6692188.0,1446450.0,8138638.0,-4841488.0,NaN,1446450.0,NaN,NaN,363855.0
27211,NaN,800248050.0,NaN,NaN,NaN,NaN,NaN,NaN,271594.0,54559.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27212,NaN,830048983.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27213,NaN,830087188.0,NaN,NaN,NaN,NaN,NaN,NaN,791068.0,72843.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27214,NaN,900007404.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-111931.0,...,NaN,14323345.0,114277.0,14437622.0,-1185538.0,NaN,114277.0,NaN,5695675.0,6664907.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27268,NaN,900510770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88315.0,521145.0,612373.0,1133518.0,-906194.0,NaN,62131.0,605.0,NaN,5336.0
27269,NaN,900637247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,26891.0,548.0,27439.0,-1109.0,NaN,548.0,0.0,567.0,27439.0
27270,NaN,900713252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,838816.0,50542875.0,51381691.0,-661184.0,NaN,4202441.0,0.0,NaN,256995.0
27271,NaN,901206570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,188917.0,-24592.0,70804.0,46212.0,-64592.0,NaN,70804.0,524.0,NaN,27614.0


In [29]:
full['event'].value_counts()

event
0.0    25931
1.0     1279
Name: count, dtype: int64

In [30]:
# This cell is to check that all missing events are of 'etapa preoperativa' to avoid biases
NULAS  =full[full['event'].isnull()]['NIT'].unique()
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
    dfs['df'+str(year)]  = pd.read_excel('Cáratula-Pymes-'+str(year)+'.xlsx')
    print(dfs['df'+str(year)][dfs['df'+str(year)]['NIT'].isin(NULAS)]['Estado actual'].value_counts())

Estado actual
EN ETAPA PREOPERATIVA    49
Name: count, dtype: int64
Estado actual
EN ETAPA PREOPERATIVA    29
Name: count, dtype: int64
Estado actual
EN ETAPA PREOPERATIVA    29
Name: count, dtype: int64
Estado actual
EN ETAPA PREOPERATIVA    20
Name: count, dtype: int64


In [31]:
full = full[-full['event'].isnull()]
full = full[full['time-event']!=2016]

In [32]:
full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'] = full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].apply(lambda x : x[0:1])

In [33]:
actividades = {
    'A': 'Agricultura, ganadería, caza, silvicultura y pesca',
    'B': 'Explotación de minas y canteras',
    'C': 'Industrias manufacturetas',
    'D': 'Suministro de electricidad, gas, vapor y aire acondicionado',
    'E': 'Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneanmiento ambiental',
    'F': 'Construcción',
    'G': 'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas',
    'H': 'Transporte y almacenamiento',
    'I': 'Alojamiento y servicios de comida',
    'J': 'Información y comunicaciones',
    'K': 'Actividades financieras y de seguros',
    'L': 'Actividades inmobiliarias',
    'M': 'Actividades profesionales,  científicas y técnicas',
    'N': 'Actividades de servicios administrativos y de apoyo',
    'O': 'Administración pública y defensa; planes de seguridad social de afiliación obligatoria',
    'P' :'Educación',
    'Q': 'Actividades de atención de la salud humana y de asistencia social',
    'R': 'Actividades artísticas, de entretenimiento y recreación',
    'S': 'Otras actividades de servicios',
    'T': 'Actividades de los hogares individuales en calidad de empleadores',
    'U': 'Actividades de organizaciones y entidades extraterritoriales'
}

In [34]:
full['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].replace(actividades, inplace=True)

In [35]:
full.to_csv("database-to-model.csv")